# 1. 层和块

① nn.Sequential 定义了一种特殊的Module。

In [1]:
# 回顾一下多层感知机
import torch
from torch import nn  #多层感知机用的
from torch.nn import functional as F  #新引入的
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
net(X)

tensor([[ 0.0718,  0.0409, -0.0064, -0.3539, -0.1537,  0.0777,  0.0086,  0.1994,
         -0.0896, -0.1302],
        [ 0.2361, -0.0898, -0.0739, -0.2010, -0.1436, -0.0394, -0.0789,  0.1622,
         -0.0725, -0.1538]], grad_fn=<AddmmBackward0>)

# 2. 自定义块

* 任何的层或者神经网络都可以认为是Module的子类
* 自定义块的两个关键函数：__init__ 和 forward

In [3]:
import torch
from torch import nn  #多层感知机用的
from torch.nn import functional as F  #新引入的
class MLP(nn.Module):
    def __init__(self): #__init__用于定义所需要的类和参数
        super().__init__()  # 调用父类的__init__函数
        self.hidden = nn.Linear(20,256) # 全连接层1：隐藏层
        self.out = nn.Linear(256,10) # 全连接层2：输出层
        
    # 定义前向函数，self指向自己，类似this指针
    # 解决给定输入怎么做输出的问题
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    
# 实例化多层感知机的层，然后在每次调用正向传播函数调用这些层
net = MLP()
X = torch.rand(2,20)
net(X)

/root/anaconda3/envs/torch_learn/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[ 0.2659, -0.2428, -0.0856,  0.0052,  0.4246,  0.0872, -0.0060, -0.1101,
         -0.0186, -0.0413],
        [ 0.1311, -0.2480, -0.1412,  0.0235,  0.3078,  0.0445,  0.0710, -0.1132,
         -0.1025, -0.0490]], grad_fn=<AddmmBackward0>)

# 3. 顺序块

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        # 接收了一个list of input arguments，用于存储liner的信息
        super().__init__()
        for block in args:
            self._modules[block] = block # block 本身作为它的key，存在_modules里面的为层，以字典的形式 
            
    def forward(self, X):
        for block in self._modules.values():
            print(block)
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
net(X)

Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[ 0.2790, -0.2059,  0.0975,  0.1640, -0.0239,  0.1685,  0.1266, -0.0727,
         -0.2308,  0.0488],
        [ 0.2175, -0.2385,  0.1060,  0.2091,  0.0537,  0.1140,  0.0370, -0.1026,
         -0.1114, -0.0465]], grad_fn=<AddmmBackward0>)

# 4. 正向传播

In [5]:
# 通过继承 nn.Module ，更加灵活的定义网络与前向计算
# 不使用Sequential，自定义一个类，可以进行大量的自定义的计算
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False) # 随机生成权重，但是不计算梯度，不参与计算
        self.linear = nn.Linear(20,20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight + 1))  # 矩阵乘法加1
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()  # 返回一个标量
    
net = FixedHiddenMLP()
X = torch.rand(2,20)
net(X)

tensor(0.0037, grad_fn=<SumBackward0>)

# 5. 混合组合块（套娃）

In [5]:
# 理解：嵌套使用，因为现有的层和网络都是nn.Module的子类
# 混合代培各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimear = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
X = torch.rand(2,20)
chimear(X)

tensor(-0.1488, grad_fn=<SumBackward0>)

# 6. 参数管理

In [8]:
# 首先关注具有单隐藏层的多层感知机
import torch
from torch import nn
 
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))  # Sequential可以理解成一个list
X = torch.rand(size=(2,4))
print(net(X))
print(net[2].state_dict()) # 访问参数，net[2]就是最后一个输出层，线性层的state就是权重和偏移
print(type(net[2].bias)) # 目标参数
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad == None) # 还没进行反向计算，所以grad为None
print(*[(name, param.shape) for name, param in net[0].named_parameters()])  # 一次性访问所有参数         
print(*[(name, param.shape) for name, param in net.named_parameters()])  # 0是第一层名字，1是ReLU，它没有参数
print(net.state_dict()['2.bias'].data) # 通过名字获取参数

tensor([[-0.1048],
        [-0.0973]], grad_fn=<AddmmBackward0>)
OrderedDict([('weight', tensor([[ 0.3125,  0.2318, -0.2194,  0.2756, -0.2195, -0.2564, -0.0820,  0.1823]])), ('bias', tensor([-0.3087]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3087], requires_grad=True)
tensor([-0.3087])
True
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
tensor([-0.3087])


# 7. 嵌套块

In [7]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1()) # f'block{i}' 可以传一个字符串名字过来，block2可以嵌套四个block1                                      
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
print(rgnet(X))
print(rgnet)

tensor([[-0.1750],
        [-0.1750]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


# 8 内置初始化

In [9]:
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))

def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) # 下划线表示把m.weight的值替换掉   
        nn.init.zeros_(m.bias)
        
net.apply(init_normal) # 会递归调用 直到net里的所有层都初始化
print(net[0].weight.data[0])
print(net[0].bias.data[0])

tensor([-0.0113,  0.0138,  0.0002, -0.0009])
tensor(0.)


In [10]:
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))

def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
print(net[0].weight.data[0]) 
print(net[0].bias.data[0])

tensor([1., 1., 1., 1.])
tensor(0.)


In [12]:
# 对某些块应用不同的初始化
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) # 宇宙的答案
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.3779, -0.3074, -0.2969, -0.5504],
        [ 0.0260, -0.1049, -0.6233, -0.2050],
        [-0.0937, -0.2387, -0.4923,  0.4747],
        [ 0.0792,  0.5286,  0.2346,  0.0479],
        [-0.0352,  0.1823, -0.1764, -0.4183],
        [-0.2254,  0.2669,  0.6601, -0.0258],
        [-0.2206,  0.3543,  0.6917,  0.5300],
        [-0.6918,  0.3496,  0.4863, -0.2998]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


# 9. 参数替换

In [14]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",*[(name, param.shape) for name, param in m.named_parameters()][0])  # 打印名字是啥，形状是啥       
        nn.init.uniform_(m.weight, -10, 10) # 均匀初始化
        m.weight.data *= m.weight.data.abs() >=  5 # 这里*=的代码相当于先计算一个布尔矩阵(先判断>=)，然后再用布尔矩阵的对应元素去乘以原始矩阵的每个元素。保留绝对值大于5的权重，不是的话就设为0

net.apply(my_init)
print(net[0].weight[:2])
net[0].weight.data[:] += 1 # 参数替换
net[0].weight.data[0,0] = 42
print(net[0].weight.data[0])

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([[-5.3670, -0.0000,  6.3498, -6.8195],
        [ 0.0000, -5.5124, -7.0872, -5.8630]], grad_fn=<SliceBackward0>)
tensor([42.0000,  1.0000,  7.3498, -5.8195])


# 10. 参数绑定

In [18]:
# 在一些层之前共享权重
# 参数绑定
shared = nn.Linear(8,8) # 1.先构造要共享的层
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))  # 第2个隐藏层和第3个隐藏层是share权重的，第一个和第四个是自己的  
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data )
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([[ 1.0000e+02,  3.1971e-01, -1.0494e-01, -1.1566e-01, -2.6466e-01,
         -1.8208e-01,  1.4149e-01, -3.4933e-01],
        [ 1.4787e-01, -1.1238e-01, -2.1118e-01, -2.1453e-01,  2.2779e-01,
          2.9798e-02,  2.7491e-01,  1.7850e-02],
        [-2.6862e-01,  2.5214e-01,  2.5834e-01, -6.3154e-02, -6.3644e-02,
         -1.2992e-01, -3.0072e-01, -2.7816e-01],
        [-3.2799e-01, -1.2887e-01, -2.5755e-01, -2.4677e-01,  1.5814e-01,
          2.5854e-01,  1.3341e-01, -3.5038e-01],
        [ 2.9017e-01, -1.4982e-01, -6.1505e-02, -2.4450e-02, -1.8523e-01,
         -2.3352e-02,  2.6686e-01, -2.9543e-01],
        [ 2.9985e-01,  3.1190e-01,  6.2135e-02,  2.2126e-01,  2.4378e-01,
          1.7160e-02, -2.3796e-01,  1.6013e-01],
        [-9.0081e-02,  4.4785e-02,  1.7597e-01, -1.2041e-01, -5.5048e-02,
          3.4668e-01, -2.7371e-01,  2.2327e-01],
        [-1.3605e-01, -1.9636e-01, -2.9743e-01,  1.9270e-01, -1.5947e-01,
        

# 11. 自定义层

In [19]:
# 与自定义网络本质相同，同样是nn.Module的子类
# 构造一个没有任何参数的自定义层
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()   # 输入减均值，使得均值变成0
    
layer = CenteredLayer()
print(layer(torch.FloatTensor([1,2,3,4,5])))

# 将层作为组件合并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())
Y = net(torch.rand(4,8))
print(Y.mean())

# 带参数的层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units)) # nn.Parameter使得这些参数加上了梯度  输入大小乘以输出大小
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
dense = MyLinear(5,3)
print(dense.weight)

# 使用自定义层直接执行正向传播计算
print(dense(torch.rand(2,5)))
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))
print(net(torch.rand(2,64)))

tensor([-2., -1.,  0.,  1.,  2.])
tensor(7.4506e-09, grad_fn=<MeanBackward0>)
Parameter containing:
tensor([[-0.5568,  1.3441,  0.1659],
        [ 0.5718,  0.1446,  0.6569],
        [-0.5675,  0.7232, -0.4598],
        [-1.2581,  0.9977, -1.6795],
        [ 2.0943,  0.4172, -0.6235]], requires_grad=True)
tensor([[0.0199, 2.7975, 0.0000],
        [0.0000, 1.7644, 0.0000]])
tensor([[3.4710],
        [2.4605]])


# 12. 读写文件

In [32]:
# 加载和保存张量（怎么存一个矩阵）
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(5)
torch.save(x, 'x-file1') # 将工程中的张量保存到文件中
x2 = torch.load('x-file1') # 将文件中的张量加载到工程中
print(x2)

#存储一个张量列表，然后把它们读回内存
y = torch.zeros(4)
torch.save([x,y],'x-files')
x2, y2 = torch.load('x-files')
print(x2)
print(y2)

# 写入或读取从字符串映射到张量的字典
mydict = {'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2 = torch.load('mydict')
print(mydict2)

tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 3, 4])
tensor([0., 0., 0., 0.])
{'x': tensor([0, 1, 2, 3, 4]), 'y': tensor([0., 0., 0., 0.])}


In [15]:
# 加载和保存模型参数
# 网络中有什么可以存的参数 - 权重
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

# 将模型的参数存储为一个叫做"mlp.params"的文件
torch.save(net.state_dict(),'mlp.params')

# 实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数
clone = MLP() # 必须要先声明一下，才能导入参数
clone.load_state_dict(torch.load("mlp.params"))
print(clone.eval()) # eval()是进入测试模式

Y_clone = clone(X)
print(Y_clone == Y)

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)
tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])
